In [ ]:
from os import name
import datetime
import plotly.graph_objects as go
from plotly import offline
import numpy as np
import numpy.typing as npt
from bache_thesis_python import fft, resample
import polars as pl
from pathlib import Path
import pprint


CURRENT_DIR = Path.cwd()
pprint.pprint(f'__name__ = {__name__}')
pprint.pprint(f'__name__ == "__main__" = {__name__ == "__main__"}')
pprint.pprint(CURRENT_DIR)

'__name__ = __main__'
'__name__ == "__main__" = True'
PosixPath('/home/iori/daxue/bache-thesis-python/analysis')


In [ ]:
PATH_TO_0630_DATA_DIR : str = "/home/iori/daxue/bache_thesis/20230630_軌道変位データ/"
SOURCE_0630_FILE :str = PATH_TO_0630_DATA_DIR + "TGdata20230630_converted.csv"
# PATH_TO_0126_DATA_DIR : str = "/home/iori/daxue/bache_thesis/20240126_軌道変位データ/"
# SOURCE_0126_FILE :str = PATH_TO_0126_DATA_DIR + "TGdata20240126_converted.csv"
OUTPUT_0630_FILE:str = PATH_TO_0630_DATA_DIR + "TGdata20230630_resampled.csv"

In [ ]:
F_BASE:float = 1/(0.25) # 元データのfreq = 4(cycle/m)
F_TRANS:float = 8*F_BASE # UP sampling後の周波数 
F_FOR_FFT:float = F_BASE # FFT用の周波数 

In [ ]:
df_source_data = pl.read_csv(SOURCE_0630_FILE).with_columns(pl.col('キロ程(meter)') - 114.5 ) # source data 読み込み,キロ程が114.5Mから始まるので0Mにオフセット
df_resampled_data_tmp : pl.DataFrame = resample(
    df_source_data,
    'キロ程(meter)',
    ['平面性(生波形)','水準(生波形)','10m弦高低(右)(狂い量)','10m弦高低(左)(狂い量)','10m弦通り(右)(生波形)','10m弦通り(左)(生波形)','軌間(生波形)'],
    F_BASE,
    F_TRANS,
    'UP',
)

Base Sampling rate is 4.0 Hz. Transformed sampling rate is 32.0 Hz.
Max Time is 67898.5541521 s
Number of sampling point is 271594. It will be 2172754 after resampling
Resampling 平面性(生波形)...
Resampling 水準(生波形)...
Resampling 10m弦高低(右)(狂い量)...
Resampling 10m弦高低(左)(狂い量)...
Resampling 10m弦通り(右)(生波形)...
Resampling 10m弦通り(左)(生波形)...
Resampling 軌間(生波形)...
Applying filter against: 平面性(生波形)...
[1.20000005 1.16257502 1.12515    ... 1.89999998 1.89999998 1.89999998]
Applying filter against: 水準(生波形)...
[ 9.19999981  9.24989988  9.29979995 ... -0.27499996 -0.33749998
 -0.40000001]
Applying filter against: 10m弦高低(右)(狂い量)...
[9.02320004 9.08986165 9.15652327 ... 5.99262743 5.92637029 5.86011314]
Applying filter against: 10m弦高低(左)(狂い量)...
[9.30285072 9.25777672 9.21270272 ... 6.00881056 5.94448831 5.88016605]
Applying filter against: 10m弦通り(右)(生波形)...
[ 4.          3.97504999  3.95009999 ... -4.12500006 -4.21250013
 -4.30000019]
Applying filter against: 10m弦通り(左)(生波形)...
[-3.         -2.98752501 -2.97

In [ ]:
df_resampled_data = resample(
    df_resampled_data_tmp,
    'キロ程(meter)',
    ['平面性(生波形)','水準(生波形)','10m弦高低(右)(狂い量)','10m弦高低(左)(狂い量)','10m弦通り(右)(生波形)','10m弦通り(左)(生波形)','軌間(生波形)'],
    F_TRANS,
    F_BASE,
    'DOWN',
)

Base Sampling rate is 32.0 Hz. Transformed sampling rate is 4.0 Hz.
Max Time is 67898.5541521 s
Number of sampling point is 2172753. It will be 271595 after resampling
Applying filter against: 平面性(生波形)...
[1.2336022  1.17699271 1.11845087 ... 1.90744458 1.90820768 1.90864328]
Applying filter against: 水準(生波形)...
[ 9.17740738  9.23665958  9.2969115  ... -0.24777789 -0.28252214
 -0.31517385]
Applying filter against: 10m弦高低(右)(狂い量)...
[9.06078468 9.11331394 9.16386395 ... 5.98547667 5.95664962 5.92913224]
Applying filter against: 10m弦高低(左)(狂い量)...
[9.37553049 9.31435418 9.25049375 ... 6.00576532 5.98048595 5.95586698]
Applying filter against: 10m弦通り(右)(生波形)...
[ 4.00929083  3.98359819  3.95776519 ... -4.12893149 -4.19072523
 -4.24613353]
Applying filter against: 10m弦通り(左)(生波形)...
[-3.00841902 -2.9960771  -2.9835708  ...  4.08515997  4.13065197
  4.17115613]
Applying filter against: 軌間(生波形)...
[ 0.40092321  0.40097829  0.40100434 ... -2.17734744 -2.19031784
 -2.20151596]
Resampling 平面性(生波形)

In [ ]:
#  [ ] 等間隔サンプリングからのズレの確認
df_compare = pl.DataFrame(
    [
        pl.Series(
            np.array([
                i*(0.25) for i in range(df_resampled_data.height)
                ]
            )
        ).alias('uniform_キロ程'),
        df_resampled_data['キロ程(meter)'].alias('resampled_キロ程'),
    ]

)

In [ ]:
df_compare = df_compare.with_columns(
    (pl.col('uniform_キロ程') - pl.col('resampled_キロ程')).alias('diff')
)
df_compare.describe()

statistic,uniform_キロ程,resampled_キロ程,diff
str,f64,f64,f64
"""count""",271595.0,271595.0,271595.0
"""null_count""",0.0,0.0,0.0
"""mean""",33949.25,33949.277076,-0.027076
"""std""",19600.716879,19600.732512,0.015632
"""min""",0.0,0.0,-0.054152
"""25%""",16974.75,16974.763538,-0.040614
"""50%""",33949.25,33949.277076,-0.027076
"""75%""",50924.0,50924.040614,-0.013538
"""max""",67898.5,67898.554152,0.0


In [ ]:
df_resampled_data.write_csv(OUTPUT_0630_FILE)